# ECE174 MiniProject 1
## Problem 1: Least Squares Classifier
## Kwok Hung Ho A15151703

In [109]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as geek  
X = loadmat("mnist.mat")
data_train = X['trainX'] # Training set digits 
data_label = X['trainY'] # Training set labels
data_test = X['testX'] # Training set digits 
data_test_label = X['testY']
inv   = np.linalg.inv
pinv  = np.linalg.pinv
solve = np.linalg.solve
rank  = np.linalg.matrix_rank
norm  = np.linalg.norm
det   = np.linalg.det
solve = np.linalg.solve
lstsq = np.linalg.lstsq

#### All the functions used are declared below:

In [208]:
def onevsall(data, label):
    H = []
    G = []
    theta = np.array([])
    aggG =  np.array([])
    for t in range(10):
        G=[]
        H=[]
        for i in range(len(data)):
            if (label[0,i] == t):
                H.append(data[i,:])
                G.append(1)
            else:
                H.append(data[i,:])
                G.append(-1)
        Gnp = np.array(G)
        aggG = np.append(aggG, Gnp)
        beta = lstsq(data, Gnp, rcond=None)[0]
        theta = np.append(theta,beta)
    theta = theta.reshape(10, 784)
    aggG = aggG.reshape(len(data),10)
    return theta, aggG

def onevsallconfusionmatrix(data, label, theta):
    confusion = np.zeros(100).reshape(10,10)
    #theta, aggG = onevsall(data, label)
    raw_label = data@(np.transpose(theta))
    a=[]
    predictlabel = []
    for i in range(len(data)):
        a = np.argmax(raw_label[i,:])
        b = label[:,i]
        confusion[a,b] = confusion[a,b]+1
        confusion = confusion.astype(int) 
        predictlabel.append(a)
    print(confusion)
    return confusion, predictlabel
    
def onevsone(data, label):
    #TO TEST USE H[i]@theta[i]=aggG[i] i from 0 to 44d
    #aggH and aggG are lists of numpy arrrays
    theta = np.array([])
    aggH = []
    aggG =  []
    #thetadict = {}
    #labeldict = {}
    for i in range(10):
        for j in range(10):
            if i<j:
                H = []
                G = []
                for x in range(len(data)):
                    if (data_label[0,x] == i):
                        H.append(data[x,:])
                        G.append(1)
                    elif (label[0,x] == j):
                        H.append(data[x,:])
                        G.append(-1)
                H = np.array(H)
                aggH.append(H)
                G = np.array(G)
                aggG.append(G)
                beta = lstsq(H, G, rcond=None)[0]
                theta = np.append(theta,beta)
                #theta = np.transpose(theta)
                #print(aggG.shape)
                #print(Gnp.shape)
                #print(theta.shape)
                #print(H.shape)
    theta = theta.reshape(45,784)
    theta = theta.T
    return theta, aggG, aggH

def onevsone45mapping():
    main_array = np.empty((0,9), int)
    init = 9
    start_val = 9
    for i in range(9):
        vals = 9 - i
        #for j in range(vals):
        zeroes = np.zeros(i)
        if i == 0:
            zeroes = np.zeros(0)
            values = np.arange(0, 9)
            init = init - 1
        else:
            values = np.arange(start_val, start_val + vals)
            start_val = start_val + init
            init = init - 1
        append_array = np.hstack([zeroes, values])
        #print(append_array)
        main_array = np.vstack([main_array, append_array])
        #print(main_array)
    return main_array

def onevsoneconfusionmatrix(data, label, theta): #theta should be obtained from data_train
    thetaij = onevsone45mapping() # maps 0-1, 0-2,...8-9, t0 0,, 1,...44
    A = geek.sign(data@theta) 
    votetable = np.zeros((len(data[:,0]),10))
    predlabel=[]
    for i in range(len(data[:,0])): # making the vote table
        for j in range(45):
            if A[i,j] ==1:
                arg = np.argwhere(thetaij==j)[0][0]
                votetable[i,arg] +=1
            elif A[i,j] ==-1:
                arg = np.argwhere(thetaij==j)[0][1]
                votetable[i,arg+1] +=1    
        predlabel.append(np.argmax(votetable[i]))
        
    confusion = np.zeros((10,10)) # Making the confusion matrix
    for i in range(len(data[:,0])):
        a = predlabel[i]
        b = label[:,i]
        confusion[a,b] = confusion[a,b]+1
        confusion = confusion.astype(int)
    #print(confusion)
    return predlabel, thetaij, confusion

## One Vs One
Obtained weights via least squares with training data and tested it on the test data. Results are below:

In [185]:
theta, aggG, aggH = onevsone(data_train, data_label)
predlabel, thetaij, confusion = onevsoneconfusionmatrix(data_test, data_test_label, theta)
print(thetaij)
print(confusion)

[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.]
 [ 0.  9. 10. 11. 12. 13. 14. 15. 16.]
 [ 0.  0. 17. 18. 19. 20. 21. 22. 23.]
 [ 0.  0.  0. 24. 25. 26. 27. 28. 29.]
 [ 0.  0.  0.  0. 30. 31. 32. 33. 34.]
 [ 0.  0.  0.  0.  0. 35. 36. 37. 38.]
 [ 0.  0.  0.  0.  0.  0. 39. 40. 41.]
 [ 0.  0.  0.  0.  0.  0.  0. 42. 43.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 44.]]
[[ 959    0    9    8    2    8    8    1    8    6]
 [   0 1122   20    2    3    7    5   18   16    5]
 [   2    3  931   14    6    3    8   16    7    1]
 [   1    3   12  930    1   31    0    3   20   12]
 [   0    0   11    1  926    8    5   11   11   29]
 [   4    1    3   21    1  788   15    1   38   10]
 [   8    3   12    2    7   18  915    0    9    0]
 [   3    1    8    8    4    2    0  954   10   22]
 [   2    2   26   20    4   22    2    2  845    5]
 [   1    0    0    4   28    5    0   22   10  919]]


The first matrix is the mapping matrix for the one vs one classifier. Using this matrix, I can correctly increment the votetable for each picture, thus showing how many votes are cast for each class. After that, I obtained the predicted labels through the argument in the votetable with the highest votes. The confusion matrix is then obtained by comparing the predicted label and the actual label.

Below is the confusion matrix obtained from running it on the training data. 

In [207]:
theta, aggG, aggH = onevsone(data_train, data_label)
predlabel, thetaij, confusion = onevsoneconfusionmatrix(data_train, data_label, theta)
print(confusion)
finderror(predlabel, data_label)

[[5811    2   52   27   14   46   27   11   33   22]
 [   2 6633   64   41   20   45   19   65  194   17]
 [  14   28 5518  110   19   35   29   56   49   16]
 [   8   22   62 5619    3  160    1    6  107   84]
 [  11    6   54    9 5570   26   27   71   44  144]
 [  21   10   20  122    9 4909   79    9  155   27]
 [  22    2   45   22   21  106 5704    1   36    3]
 [   4    8   36   48   17    7    0 5874   26  130]
 [  29   23   92   91    6   67   31    8 5147   37]
 [   1    8   15   42  163   20    1  164   60 5469]]
number of errors: 3746
error rate is: 0.062433333333333334


## Analyzing the error:
This error is for theta found from the training set applied to the test set. (where theta is the weights)

In [175]:
predlabel, thetaij, confusion = onevsoneconfusionmatrix(data_test, data_test_label, theta) 
finderror(predlabel, data_test_label)

number of errors: 711
error rate is: 0.0711


The error seen above is about 0.0711, corresponding to 7.1%. The error represents the results of the weights obtained from training data on the test data. 

# One Vs All

In [205]:
theta, aggG = onevsall(data_train, data_label)
confusion, predictlabel = onevsallconfusionmatrix(data_train, data_label, theta)
finderror(predictlabel, data_label)

[[5665    1   93   42    9  144  107   53   82   65]
 [   8 6514  258  142   97   70   66  187  527   60]
 [  19   36 4802  183   56   30   65   43   58   25]
 [  18   16  149 5206    8  520    1   57  223  113]
 [  26   10  101   29 5125   82   59  153  123  355]
 [  43   29   11   92   49 3786   81    9  234    8]
 [  73   15  237   58   48  196 5491    3   50    4]
 [   3   13   84  109   23   38    0 5391   21  488]
 [  62  103  203  139   83  404   46   18 4358   53]
 [   6    5   20  131  344  151    2  351  175 4778]]
number of errors: 8884
error rate is: 0.14806666666666668


#### The confusion matrix above is for the training data. 

In [206]:
theta, aggG = onevsall(data_train, data_label)
confusion, predictlabel = onevsallconfusionmatrix(data_test, data_test_label, theta)
finderror(predictlabel, data_test_label)

[[ 942    0   17    4    0   20   17    5   17   18]
 [   0 1107   56   15   23   17    9   38   54   10]
 [   2    2  809   26    6    2   10   18    9    2]
 [   2    2   28  887    3   84    0    8   32   15]
 [   1    1   16    2  872   19   21   20   27   72]
 [   7    1    0   14    5  624   20    0   42    1]
 [  15    5   42    9   10   22  872    1   15    1]
 [   2    2   21   21    2   13    0  877   12   76]
 [   7   15   39   21   13   69    9    3  743   13]
 [   2    0    4   11   48   22    0   58   23  801]]
number of errors: 1466
error rate is: 0.1466


#### The confusion matrix above is for the test data. 


## Performance Evaluation:
The error for the one-vs-all classifier has about 14.5% error rate while the one-vs-one classifier has about a 7.1% error rate for the test data. The one-vs-one classifier seems to out perform the one-vs-all classifier by twice the ammount. 

Considering the test data:
For the one-vs-one classifier, the number 1 seems to be the easiest to recognizes with 1122 correct guesses, while the most difficult to recognize is the number 5 with 788 correct guesses. 

For the one-vs-all classifier, the easiset to recognize seems to be the number 1 as well with 1107 correct guesses, and the most difficult to guess is also the number 5 with 624 correct guesses. 

It seems that numbers that look similar such as 5 and 8 have a lower amount of correct guesses, while numbers such as 1 or 0 have the best guesses. The one-vs-one classifier might outperform the one-vs-all classifier because it directly compares 2 numbers while the one-vs-all compares it to a bundle of numbers. The binary classification as such is more certain when it is one-to-one, since the appearance of multiple similar numbers may less affect the direct comparison